In [34]:
import os
import piexif  # maybe not needed
import pandas as pd
from fractions import Fraction
from PIL import Image  # maybe not needed
from datetime import datetime, timedelta
import subprocess

In [63]:
# Helper methods


def make_photos_df(folder_path, file_extension='jpg'):
    all_files_list = os.listdir(folder_path)
    photos_files_list = [file for file in all_files_list if file.lower().split('.')[-1] == file_extension]
    photos_datetime_list = []
    for filename in photos_files_list:
        datetime = piexif.load(folder_path + '\\' + filename)['Exif'][36867].decode(encoding='UTF-8')
        photos_datetime_list.append(datetime)

    photos_dict = {'filename': photos_files_list, 'datetime': photos_datetime_list}
    photos_df = pd.DataFrame.from_dict(photos_dict)
    photos_df['datetime'] = pd.to_datetime(photos_df['datetime'], format='%Y:%m:%d %H:%M:%S')

    return photos_df


def make_location_history_df(csv_file_path, delete_miliseconds=True):
    ''' Temporary solution with .csv file '''
    location_history_df = pd.read_csv(csv_file_path)
    if delete_miliseconds:
        location_history_df['datetime'] = location_history_df['datetime'].apply(lambda x: x.split('.')[0])
    location_history_df['datetime'] = pd.to_datetime(location_history_df['datetime'], format='%Y-%m-%d %H:%M:%S')
    return location_history_df


def give_coordinate(coordinate_return, photo_datetime, location_df, time_column, lat_column, long_column):
    location_df['new_time_diff'] = abs(location_df[time_column] - photo_datetime)
    min_row_index = location_df['new_time_diff'].idxmin()

    if coordinate_return == 'latitude':
        return location_df[lat_column].loc[min_row_index]
    elif coordinate_return == 'longitude':
        return location_df[long_column].loc[min_row_index]
    else:
        return None


def give_latitude(photo_datetime):
    return give_coordinate(
        coordinate_return='latitude',
        photo_datetime=photo_datetime,
        location_df=location_history_df,
        time_column='datetime',
        lat_column='latitude',
        long_column='longitude'
    )


def give_longitude(photo_datetime):
    return give_coordinate(
        coordinate_return='longitude',
        photo_datetime=photo_datetime,
        location_df=location_history_df,
        time_column='datetime',
        lat_column='latitude',
        long_column='longitude'
    )


def to_deg(value, loc):
    """convert decimal coordinates into degrees, munutes and seconds tuple
    Keyword arguments: value is float gps-value, loc is direction list ["S", "N"] or ["W", "E"]
    return: tuple like (25, 13, 48.343 ,'N')
    """
    if value < 0:
        loc_value = loc[0]
    elif value > 0:
        loc_value = loc[1]
    else:
        loc_value = ""
    abs_value = abs(value)
    deg =  int(abs_value)
    t1 = (abs_value-deg)*60
    min = int(t1)
    sec = round((t1 - min)* 60, 5)
    return (deg, min, sec, loc_value)


def change_to_rational(number):
    """convert a number to rantional
    Keyword arguments: number
    return: tuple like (1, 2), (numerator, denominator)
    """
    f = Fraction(str(number))
    return (f.numerator, f.denominator)


def change_datetime(path, forward=True, years=0, months=0, days=0, hours=0, minutes=0, seconds=0):
    date_diff_list = [str(years), str(months), str(days)]
    time_diff_list = [str(hours), str(minutes), str(seconds)]
    datetime_diff = ':'.join(date_diff_list) + ' ' + ':'.join(time_diff_list)
    
    direction_operator = '+=' if forward==True else '-='
    command_to_run = ['exiftool', '-AllDates' + direction_operator + datetime_diff, path, '-overwrite_original']
    proc = subprocess.run(command_to_run, encoding='utf-8', stdout=subprocess.PIPE)
    for line in proc.stdout.split('\n'):
        print(line)
        
        
def pretty_print_dict(dictionary, indent=0):
   for key, value in dictionary.items():
      print('\t' * indent + str(key))
      if isinstance(value, dict):
         pretty_print_dict(value, indent+1)
      else:
         print('\t' * (indent+1) + str(value))

In [ ]:
# Check exif of some image to test
image_path = 'G:\Moje obrazy\Jordania - czerwiec 2019\Aparat Panasonic — kopia\P1150302.JPG'
exif_dict = piexif.load(image_path)
pretty_print_dict(dictionary=exif_dict)

In [66]:
# Add some progress bar
change_datetime(path='G:\Moje obrazy\Jordania - czerwiec 2019\Aparat Panasonic — kopia', forward=True, minutes=50)

    1 directories scanned
 1089 image files updated

